In [ ]:
class RangeBar(object):

    def __init__(self, o: float, r: float):
        self.o = o
        self.h = -1
        self.l = 1e12
        self.c = o
        self.range = r

    # handle new value, return new bar or none
    def handle(self, v: float):
        self.c = v
        if v > self.h:
            self.h = v
        if v < self.l:
            self.l = v

        if abs(v - self.o) >= self.range:
            return RangeBar(v, self.range)
        else:
            return None

    def show(self):
        print(self.o, self.h, self.l, self.c)

In [ ]:
import random
import pandas as pd

#ret = pd.DataFrame({"o": s.o, "h":s.h, "l": s.l, "c": s.c}, index = [0])

r = []
for i in range(100):
    new_value = random.randint(30, 80)
    r.append(new_value)
 
def make_range_bar(aim, window):
    ret = pd.DataFrame({}, columns=['o', 'h', 'l', 'c'])
    s = RangeBar(aim[0], window)
    arr = []
    arr.append(s)
    for i in aim:
        maybe_bar = arr[-1].handle(i)
        if maybe_bar is not None:
            ret.loc[len(ret.index)] = [arr[-1].o, arr[-1].h, arr[-1].l, arr[-1].c]
            arr.append(maybe_bar)
    return ret

ret = make_range_bar(r, 10)

In [ ]:
ret

In [ ]:
def show_df(hist):
    import plotly
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = go.Figure(data=go.Candlestick(
        x=hist.index,
        open=hist['o'],
        high=hist['h'],
        low=hist['l'],
        close=hist['c'],
    ))
    #return fig.to_html(full_html=False, include_plotlyjs="cdn") # for IDP
    fig.show() # for jupyter

In [ ]:
show_df(ret)

In [ ]:
import pandas as pd
all_df = pd.read_parquet("../data/2023-01-03-08.tick")
rb_df = all_df[all_df['symbol'] == "rb2305"]

rb_p = list(rb_df.lastPrice)

rb_rb = make_range_bar(rb_p, 5.0)

rb_df.columns





In [ ]:
show_df(make_range_bar(rb_p, 10))

In [ ]:
def make_clear_df(file, symbol):
    all_df = pd.read_parquet(file)
    aim_df = all_df[all_df['symbol'] == symbol]
    aim_df.loc[:, 'v'] = aim_df.loc[:, 'volume'].diff()
    columns = []
    f_columns = []
    for i in range(5):
        aim_df.loc[:, 'av'+str(i)] = aim_df.loc[:, 'askVolume1'].shift(i)
        aim_df.loc[:, 'bv'+str(i)] = aim_df.loc[:, 'bidVolume1'].shift(i)
        aim_df.loc[:, 'v'+str(i)] = aim_df.loc[:, 'v'].shift(i)
        columns.append('av' + str(i))
        columns.append("bv" + str(i))
        columns.append("v" + str(i))
    for i in range(60):
        aim_df.loc[:, 'p'+str(i)] = aim_df.loc[:, 'lastPrice'].shift(i) - aim_df.loc[:, 'lastPrice']
        columns.append("p" + str(i))
    for i in range(30):
        aim_df.loc[:, 'f'+str(i)] = aim_df.loc[:, 'lastPrice'].shift(-i) - aim_df.loc[:, 'lastPrice']
        columns.append("f" + str(i))
        f_columns.append("f" + str(i))
    hist = aim_df.loc[:, columns].dropna()
    hist.loc[:, 'min'] = hist[f_columns].apply(min, axis=1)
    hist.loc[:, 'max'] = hist[f_columns].apply(max, axis=1)
    return hist

t = make_clear_df("../data/2023-01-12-12.tick", 'rb2305')

In [ ]:
t['target']

In [ ]:
t

In [ ]:
",".join(t.columns)

In [ ]:

# kvm 计算起来太慢，所以不用它
'''
from sklearn import svm

classifier = svm.SVC(kernel='linear')  # Linear Kernel
feature = t[t.columns.drop('target')]
classifier.fit(feature, t['target'])

## write evaluation
from sklearn.metrics import classification_report

y_pre = classifier.predict(feature)
ret = classification_report(t['target'], y_pre, output_dict=True)
df = pd.DataFrame(ret)

df
'''

In [ ]:
t[['p0','f1','f2','f29', 'min', 'max']]

In [ ]:
x = pd.DataFrame({'x': range(10)})
x['y'] = x['x'].shift(-2)
x['mm'] = x[['x', 'y']].apply(max, axis=1)
x

In [35]:
t.head()

,av0,bv0,v0,av1,bv1,v1,av2,bv2,v2,av3,...,f22,f23,f24,f25,f26,f27,f28,f29,min,max
80933,107,91,81.0,45.0,138.0,80.0,14.0,241.0,61.0,48.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,3.0,-1.0,3.0
81192,169,42,74.0,107.0,91.0,81.0,45.0,138.0,80.0,14.0,...,3.0,2.0,3.0,3.0,3.0,3.0,4.0,5.0,0.0,5.0
81440,32,136,63.0,169.0,42.0,74.0,107.0,91.0,81.0,45.0,...,2.0,3.0,3.0,3.0,3.0,4.0,5.0,5.0,0.0,5.0
81710,212,67,114.0,32.0,136.0,63.0,169.0,42.0,74.0,107.0,...,3.0,3.0,3.0,3.0,4.0,5.0,5.0,5.0,0.0,5.0
81979,184,123,53.0,212.0,67.0,114.0,32.0,136.0,63.0,169.0,...,3.0,3.0,3.0,4.0,5.0,5.0,5.0,5.0,0.0,5.0
